# Week_1 Assignment_2: Важность признаков в Решающих деревьях

https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/DVbEI/vazhnost-priznakov

Данное задание основано на материалах лекций по логическим методам и направлено на знакомство c решающими деревьями (Decision Trees).

Вы научитесь:
- обучать решающие деревья
- находить наиболее важные для них признаки

#### Введение
Решающие деревья относятся к классу логических методов. Их основная идея состоит в объединении определенного количества простых решающих правил, благодаря чему итоговый алгоритм является интерпретируемым. Как следует из названия, решающее дерево представляет собой бинарное дерево, в котором каждой вершине сопоставлено некоторое правило вида "j-й признак имеет значение меньше b". В листьях этого дерева записаны числа-предсказания. Чтобы получить ответ, нужно стартовать из корня и делать переходы либо в левое, либо в правое поддерево в зависимости от того, выполняется правило из текущей вершины или нет.

Одна из особенностей решающих деревьев заключается в том, что они позволяют получать важности всех используемых признаков. Важность признака можно оценить на основе того, как сильно улучшился критерий качества благодаря использованию этого признака в вершинах дерева.

#### Данные
В этом задании мы вновь рассмотрим данные о пассажирах Титаника. Будем решать на них задачу классификации, в которой по различным характеристикам пассажиров требуется предсказать, кто из них выжил после крушения корабля. 

#### Реализация в Scikit-Learn
В библиотеке scikit-learn решающие деревья реализованы в классах sklearn.tree.DecisionTreeСlassifier (для классификации) и sklearn.tree.DecisionTreeRegressor (для регрессии). Обучение модели производится с помощью функции fit. 

Пример использования:

    import numpy as np
    from sklearn.tree import DecisionTreeClassifier
    X = np.array([[1, 2], [3, 4], [5, 6]])
    y = np.array([0, 1, 0])
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
  
В этом задании вам также потребуется находить важность признаков. Это можно сделать, имея уже обученный классификатор:

    importances = clf.feature_importances_

Переменная importances будет содержать массив "важностей" признаков. Индекс в этом массиве соответствует индексу признака в данных.

Стоит обратить внимание, что данные могут содержать пропуски. Pandas хранит такие значения как nan (not a number). Для того, чтобы проверить, является ли число nan'ом, можно воспользоваться функцией np.isnan.

    np.isnan(X)

#### Материалы
- Подробнее про решающие деревья в sklearn: http://scikit-learn.org/stable/modules/tree.html
- Работа с пропущенными значениями в pandas: http://pandas.pydata.org/pandas-docs/stable/missing_data.html
- Подробнее о деревьях и их построении: https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture07-trees.pdf

#### Инструкция по выполнению
1. Загрузите выборку из файла titanic.csv с помощью пакета Pandas.
2. Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).
3. Обратите внимание, что признак Sex имеет строковые значения.
4. Выделите целевую переменную — она записана в столбце Survived.
5. В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки. 
6. Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).
7. Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен). 

## Ответы на вопросы задания

In [1]:
# load libraries
import pandas as pd
import numpy as np

### 1. Загрузите выборку из файла titanic.csv с помощью пакета Pandas

In [2]:
# load data from file
data = pd.read_csv('titanic.csv', index_col='PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.shape

(891, 11)

### 2. Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).

In [4]:
# Survived тоже надо оставить, иначе после удаления Nan-строк не будет совпадать размерность
columns_list = ['Name', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked']
df_Xy = data.drop(columns=columns_list, inplace=False)
df_Xy

,Survived,Pclass,Sex,Age,Fare
PassengerId,,,,,
1,0,3,male,22.0,7.2500
2,1,1,female,38.0,71.2833
3,1,3,female,26.0,7.9250
4,1,1,female,35.0,53.1000
5,0,3,male,35.0,8.0500
...,...,...,...,...,...
887,0,2,male,27.0,13.0000
888,1,1,female,19.0,30.0000
889,0,3,female,NaN,23.4500


### 3. Обратите внимание, что признак Sex имеет строковые значения

In [5]:
# что ж, заменим строковые значения на цифровые
df_Xy.replace(['male', 'female'], [1, 0], inplace=True)
df_Xy

,Survived,Pclass,Sex,Age,Fare
PassengerId,,,,,
1,0,3,1,22.0,7.2500
2,1,1,0,38.0,71.2833
3,1,3,0,26.0,7.9250
4,1,1,0,35.0,53.1000
5,0,3,1,35.0,8.0500
...,...,...,...,...,...
887,0,2,1,27.0,13.0000
888,1,1,0,19.0,30.0000
889,0,3,0,NaN,23.4500


### 5. В данных есть пропущенные значения — удалите их из выборки
Например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки и удалите их из выборки

In [64]:
df_Xy.dropna(axis=0, how='any', inplace=True)
df_Xy

,Survived,Pclass,Sex,Age,Fare
PassengerId,,,,,
1,0,3,1,22.0,7.2500
2,1,1,0,38.0,71.2833
3,1,3,0,26.0,7.9250
4,1,1,0,35.0,53.1000
5,0,3,1,35.0,8.0500
...,...,...,...,...,...
886,0,3,0,39.0,29.1250
887,0,2,1,27.0,13.0000
888,1,1,0,19.0,30.0000


### 4. Выделите целевую переменную — она записана в столбце Survived

In [68]:
# выделяем входной массив параметров
X = df_X.values[:,1:]
print(X)

[[ 3.      1.     22.      7.25  ]
 [ 1.      0.     38.     71.2833]
 [ 3.      0.     26.      7.925 ]
 ...
 [ 1.      0.     19.     30.    ]
 [ 1.      1.     26.     30.    ]
 [ 3.      1.     32.      7.75  ]]


In [80]:
# выделяем вектор целевой переменной
y = df_X.values[:,0]
print(y)

[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.

### 6. Обучите решающее дерево 
с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).

In [81]:
# делаем как в инструкции
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=241)
clf = clf.fit(X, y)

### 7. Вычислите важности признаков и найдите два признака с наибольшей важностью
Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

In [74]:
print(clf.feature_importances_)

[0.14000522 0.30051221 0.2560461  0.30343647]


### Ответ: Sex, Fare

## Что не получилось / надо сделать:

1. Не понял зачем понадобилось менять значения столбца "Sex" на 0/1. То есть, формально понятно: иначе не получил бы np.array для Х. Но сомнение осталось: ведь решающие деревья могут работать с любыми форматами параметров? Значит, это ограничения конкретной библиотеки? А есть ли реализации, позволяющие не использовать np.array, а скармливать данные "как есть"?

2. Не вывел картинку дерева решений. Вариант, указанный в [мануале](http://scikit-learn.org/stable/modules/tree.html), выводит очень мелкую картинку. Как ее увеличить - не разобрался.

    tree.plot_tree(clf)   # рабтает плохо, картинка очень мелкая

3. В документации есть вариант экспорта дерева в Graphviz и затем отрисовка красивого дерева: 

    import graphviz 
    dot_data = tree.export_graphviz(clf, out_file=None) 
    graph = graphviz.Source(dot_data) 
    graph.render("iris") 
    dot_data = tree.export_graphviz(clf, out_file=None, 
                      feature_names=iris.feature_names,  
                      class_names=iris.target_names,  
                      filled=True, rounded=True,  
                      special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph 

Для этого надо сначала установить Graphviz командой (так в мануале): 

    !conda install python-graphviz                
    
Эта установка у меня зашла в бесконечный режим. Возможно, надо разобраться с синтаксисом конды. Например, работающая команда по геопи выглядела так:

    !conda install -c conda-forge geopy --yes

4. Кроме класификации надо попробовать регрессию. Хотябы со сгенерированной функцией "синус+шум", как в мануале. 

5. Читай 1.10.5. Tips on practical use - там много полезного